In [15]:
import pandas as pd
import pandera as pa

In [16]:
valores_ausentes = ['***', '****' , '*****', '####','NULL']
df = pd.read_csv("ocorrencia.csv", sep = "," , parse_dates=["ocorrencia_dia"], dayfirst= True, na_values= valores_ausentes)
df.head(5)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


In [19]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia": pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,2), nullable= True),
        "ocorrencia_aerodromo": pa.Column(pa.String,  nullable= True),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

In [20]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [22]:
filtro = df.ocorrencia_aerodromo.isnull()
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
...,...,...,...,...,...,...,...,...,...
6092,80236,80236,ACIDENTE,POCONÉ,MT,NaN,2021-08-01,22:00:00,0
6093,80227,80227,ACIDENTE,UMUARAMA,PR,NaN,2021-08-02,14:30:00,0
6096,80229,80229,INCIDENTE,ITAGUAÍ,RJ,NaN,2021-08-05,15:30:00,0
6099,80233,80233,ACIDENTE,POCONÉ,MT,NaN,2021-08-08,12:30:00,0


In [23]:
df.count()

codigo_ocorrencia           6114
codigo_ocorrencia2          6114
ocorrencia_classificacao    6114
ocorrencia_cidade           6114
ocorrencia_uf               6112
ocorrencia_aerodromo        3664
ocorrencia_dia              6114
ocorrencia_hora             6113
total_recomendacoes         6114
dtype: int64

In [24]:
#ocorrência com mais de 10 recomendações
filtro = df.total_recomendacoes >10
df.loc[filtro,['ocorrencia_cidade', 'total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
63,SÃO PAULO,11
326,RIO DE JANEIRO,19
530,SÃO PAULO,11
727,ITÁPOLIS,11
872,RECIFE,23
884,SÃO JOSÉ DOS PINHAIS,11
1062,UBERLÂNDIA,20
1972,BRASÍLIA,11
2797,SANTOS,13
3932,VITÓRIA,12


In [25]:
#ocorrencia cuja classificação == INCIDENTE GRAVE
filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
29,39709,39709,INCIDENTE GRAVE,CURITIBA,PR,SBBI,2010-01-23,16:36:00,2
35,39487,39487,INCIDENTE GRAVE,PALMAS,TO,SBPJ,2010-01-26,16:50:00,0
...,...,...,...,...,...,...,...,...,...
6090,80221,80221,INCIDENTE GRAVE,MARAÚ,BA,SIRI,2021-07-31,13:15:00,0
6095,80228,80228,INCIDENTE GRAVE,ELDORADO DO SUL,RS,SIXE,2021-08-05,15:55:00,0
6097,80231,80231,INCIDENTE GRAVE,CAXIAS DO SUL,RS,SBCX,2021-08-06,16:59:00,0
6098,80248,80248,INCIDENTE GRAVE,CHAPADÃO DO SUL,MS,SSCD,2021-08-06,20:00:00,0


In [26]:
#ocorrencia cuja a classificação == INCIDENTE GRAVE e estado SP
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
136,40270,40270,INCIDENTE GRAVE,PIRASSUNUNGA,SP,SDPY,2010-04-15,20:15:00,0
171,40620,40620,INCIDENTE GRAVE,GUARULHOS,SP,SBGR,2010-05-06,15:50:00,0
219,41411,41411,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-06-08,19:30:00,2
297,42289,42289,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-07-29,20:06:00,0
314,42323,42323,INCIDENTE GRAVE,SÃO JOSÉ DO RIO PRETO,SP,SBSR,2010-08-05,12:01:00,0
...,...,...,...,...,...,...,...,...,...
5949,80056,80056,INCIDENTE GRAVE,FERNANDÓPOLIS,SP,NaN,2021-04-28,11:20:00,0
6000,80114,80114,INCIDENTE GRAVE,LINS,SP,SWXQ,2021-06-05,19:43:00,0
6007,80124,80124,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2021-06-10,20:30:00,0
6081,80200,80200,INCIDENTE GRAVE,BRAGANÇA PAULISTA,SP,SDVH,2021-07-25,12:25:00,0


In [27]:
#ocorrencia cuja a classificação == INCIDENTE GRAVE ou estado SP
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 | filtro2]


,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
...,...,...,...,...,...,...,...,...,...
6097,80231,80231,INCIDENTE GRAVE,CAXIAS DO SUL,RS,SBCX,2021-08-06,16:59:00,0
6098,80248,80248,INCIDENTE GRAVE,CHAPADÃO DO SUL,MS,SSCD,2021-08-06,20:00:00,0
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0


In [28]:
#ocorrencia cuja a classificação == INCIDENTE GRAVE ou classificação == INCIDENTE e o estado SP
filtro1 = (df.ocorrencia_classificacao == 'INCIDENTE GRAVE') | (df.ocorrencia_classificacao == 'INCIDENTE')
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 | filtro2]


,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
...,...,...,...,...,...,...,...,...,...
6108,80244,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0
6109,80245,80245,INCIDENTE,RECIFE,PE,SBRF,2021-08-15,15:05:00,0
6110,80247,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0
6111,80251,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0


In [34]:
#ocorrencia cuja a cidade contém (em qualquer parte do conteúdo) os caracteres MA
filtro =  df.ocorrencia_cidade.str.contains('MA | AL')
df.loc[filtro]


,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
110,40089,40089,ACIDENTE,VILA BELA DA SANTÍSSIMA TRINDADE,MT,NaN,2010-03-28,15:10:00,0
130,40258,40258,INCIDENTE,MONTE ALEGRE,PA,NaN,2010-04-12,18:10:00,0
131,40327,40327,INCIDENTE,PORTO ALEGRE,RS,NaN,2010-04-12,13:13:00,0
133,40248,40248,INCIDENTE,PORTO ALEGRE,RS,SBPA,2010-04-13,18:47:00,0
156,40572,40572,ACIDENTE,PORTO ALEGRE,RS,SBPA,2010-04-28,14:10:00,0
...,...,...,...,...,...,...,...,...,...
5849,79912,79912,INCIDENTE,PORTO ALEGRE,RS,SBPA,2021-02-23,16:12:00,0
5851,79920,79920,INCIDENTE,PORTO ALEGRE,RS,SBPA,2021-02-24,02:57:00,0
5906,80023,80023,INCIDENTE,PORTO ALEGRE,RS,SBPA,2021-03-29,18:24:00,0
5916,80027,80027,INCIDENTE,PORTO ALEGRE,RS,SBPA,2021-04-07,19:47:00,0


In [41]:
#ocorrencia cuja a cidade termine com os caracteres MA
filtro =  df.ocorrencia_cidade.str[-1] == 'M'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
36,39850,39850,INCIDENTE,BELÉM,PA,SBJC,2010-01-26,14:30:00,0
40,39158,39158,INCIDENTE,BELÉM,PA,NaN,2010-01-28,16:00:00,0
92,40322,40322,INCIDENTE,BELÉM,PA,SBJC,2010-03-15,18:15:00,0
93,40307,40307,INCIDENTE,BELÉM,PA,SBJC,2010-03-16,20:00:00,0
...,...,...,...,...,...,...,...,...,...
5922,80019,80019,ACIDENTE,NOVA MUTUM,MT,NaN,2021-04-12,11:00:00,0
5937,80040,80040,INCIDENTE,BELÉM,PA,SBBE,2021-04-20,17:10:00,0
5974,80085,80085,INCIDENTE GRAVE,SANTARÉM,PA,SNCJ,2021-05-13,19:30:00,0
6062,80190,80190,INCIDENTE,SANTARÉM,PA,SBSN,2021-07-11,21:30:00,0


In [45]:
#ocorrencia do ano de 2015
filtro = (df.ocorrencia_dia.dt.year == 2015) & (df.ocorrencia_dia.dt.month == 12) & (df.ocorrencia_dia.dt.day > 2 ) & (df.ocorrencia_dia.dt.day < 9)
df.loc[filtro]


,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3434,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3435,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3436,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
3437,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3438,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0
3440,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0
3441,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0
